# Klasyfikator minimalno-odległościowy

Notatnik ten zawiera implementacją klasyfikatora minimalno-odległościowego w numpy. Jest on podobny do omawianego na ćwiczeniach klasyfikatora kNN, jednak znacznie od niego prostszy.

# Minimum distance classifier

This notebook contains an implementation of the minimum distance classifier in numpy. It is similar to the kNN classifier discussed in the exercises, but much simpler than it.

**1\. Generacja danych.**

Tym razem, zamiast wczytywać dane, wygeneruj je z wykorzystaniem funkcji [make_blobs](http://scikit-learn.org/0.16/modules/generated/sklearn.datasets.make_blobs.html#sklearn.datasets.make_blobs). 

Ustaw ilość próbek na `500`, a ilość grup (centrów) na `3`. Ponadto, aby twoje rysunki były analogiczne jak przedstawione poniżej, ważne jest ustawienie `random_state=2`.


**1 \. Data generation.**

This time, instead of loading data, generate it using the function [make_blobs](http://scikit-learn.org/0.16/modules/generated/sklearn.datasets.make_blobs.html#sklearn.datasets.make_blobs).

Set the number of samples to `500` and the number of groups (centers) to` 3`. In addition, for your drawings to be analogous to those shown below, it is important to set `random_ state = 2`.

In [ ]:
%matplotlib inline
import numpy as np 
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import datasets 
from sklearn.neighbors import NearestCentroid
import matplotlib.pyplot as plt

#metoda generujaca dane treningowe i testowe
from sklearn.datasets import make_blobs

blobs_data = None
blobs_target = None

### BEGIN SOLUTION
blobs_data, blobs_target = make_blobs(n_samples = 500, n_features=2, centers=3, random_state=2)
### END SOLUTION


In [ ]:
assert blobs_data.shape == (500, 2)
assert blobs_data.shape[0] == blobs_target.shape[0]

In [ ]:
### BEGIN HIDDEN TESTS
X, Y  = make_blobs(n_samples = 500, n_features=2, centers=3, random_state=2)
np.testing.assert_array_equal(blobs_data,  X)
### END HIDDEN TESTS

**Wizualizacja wygenerowanych danych**

**Visualization of generated data**

In [ ]:
plt.scatter(blobs_data[:,0], blobs_data[:,1],c= np.ravel(blobs_target))

**2\. Normalizacja danych wejściowych**

Przed rozpoczęciem należy zapewnić bezpośrednią porównywalność cech, czyli dokonać normalizacji. Skorzystaj ze wzoru z [Definicji 2](http://books.icse.us.edu.pl/runestone/static/ai/IstotaUczeniaNadzorowanegoINienadzorowanego/ReprezentacjaWzorcow.html#przetwarzanie-wstepne) 
 

 

**2 \. Normalization of input data**

Prior to commencing, direct comparability of characteristics should be ensured, i.e. normalized. 

In [ ]:
def normalize(v):
### BEGIN SOLUTION
    return (v-np.mean(v,axis=0))/np.std(v,axis=0)
### END SOLUTION

In [ ]:
blobs_data_normalized = normalize(blobs_data)

np.testing.assert_array_almost_equal(blobs_data_normalized[100], np.array([0.39173587, 0.67031484]))

*Wykres po normalizacji*

*Graph after normalization*

In [ ]:
plt.scatter(blobs_data_normalized[:,0], blobs_data_normalized[:,1],c= np.ravel(blobs_target))

**3\. Podział na dane treningowe i testowe.**

W sposób losowy podziel wygenerowane dane na zriór uczący i testowy w proporcji 70% do 30%. Ustaw `random_state=123`

**3 \. Division into training and test data.**

Randomly split the generated data into a learning and test grid in a 70% to 30% ratio. Set `random_state = 123`

In [ ]:
train, test, train_labels, test_labels = None, None, None, None

### BEGIN SOLUTION
train, test, train_labels, test_labels = \
                        train_test_split(blobs_data_normalized, blobs_target, test_size=0.3, random_state=123)
###END SOLUTION

In [ ]:
assert np.shape(train)[0] == 0.7 * np.shape(blobs_data_normalized)[0]
assert np.shape(test)[0] == np.shape(blobs_data_normalized)[0] - np.shape(train)[0]

**4\. Wyznacz obiekt typowy dla każdej klasy, czyli prezentowana przez jeden typowy dla niej obiekt.**

W roli tego reprezentanta najczęściej występuje środek (średnia arytmetyczna) obiektów z danej klasy. 

**4 \. Designate a typical object for each class, i.e. represented by one typical object for it.**

The role of this representative is most often the center (arithmetic mean) of objects from a given class.

In [ ]:
class_means = None
### BEGIN SOLUTION
class_means = np.vstack([ np.mean( train[np.ravel(train_labels==i)], axis=0) for i in np.unique(train_labels) ])
###END SOLUTION

In [ ]:
assert class_means[0,1]==-1.3155046512783324

In [ ]:
### BEGIN HIDDEN TESTS
np.testing.assert_array_almost_equal(class_means, \
                 np.vstack([ np.mean( train[np.ravel(train_labels==i)], axis=0) for i in np.unique(train_labels) ]))
### END HIDDEN TESTS

In [ ]:
plt.scatter(train[:,0], train[:,1],c= np.ravel(train_labels))
plt.plot(class_means[0,0], class_means[0,1], 'ro',markersize=10)
plt.plot(class_means[1,0], class_means[1,1], 'ro',markersize=10)
plt.plot(class_means[2,0], class_means[2,1], 'ro',markersize=10)

**5\. Odległość wektorów testowych od reprezentantów klas**

Dla każdej danej testowej, reprezentowanej przez wektor cech wyznacz odległość od reprezentantów klas


**5 \. Distance of test vectors from class representatives**

Determine the distance from class representatives for each test data represented by the feature vector

In [ ]:
def distances(x):
    ### BEGIN SOLUTION
    return np.stack([ np.sqrt(np.sum( (x - class_means[i])**2, axis=1)) for i in range(len(class_means)) ]).T
    
    ###END SOLUTION

In [ ]:
assert distances(test).shape == (150,3)
assert distances(test)[1,1]==1.7886834573805668

In [ ]:
### BEGIN HIDDEN TESTS
np.testing.assert_array_almost_equal(distances(test), \
                 np.stack([ np.sqrt(np.sum( (test - class_means[i])**2, axis=1)) for i in range(len(class_means)) ]).T)
### END HIDDEN TESTS

**6\. Funkcja klasyfikująca**

Dla danych testowych wyznacza klasę, dla której obiekt ją reprezentujący jest najbliższy.

**6 \. Classifying function**

For test data, it determines the class for which the object representing it is closest.

In [ ]:
def classify(x):
    ### BEGIN SOLUTION
    return np.argmin(distances(x), axis=1)
    
    ###END SOLUTION

In [ ]:
assert classify(test).shape == (150,)
assert classify(test)[100]==2

In [ ]:
### BEGIN HIDDEN TESTS
np.testing.assert_array_almost_equal(classify(test), \
                 np.argmin(distances(test), axis=1))
### END HIDDEN TESTS

**7\. Jaki jest procent poprawych odpowiedzi?**

**7 \. What is the percentage of correct answers?**

In [ ]:
score = None
### BEGIN SOLUTION
score = np.sum(classify(test)==test_labels)/len(test_labels)
    
###END SOLUTION

In [ ]:
np.testing.assert_almost_equal(score, 0.9666666666666667)

**8\. Stwórz klasę `MinimumDistanceClassifier` analogiczną do klasy `NearestCentroid` z `sklearn`**

**8 \. Create a `MinimumDistanceClassifier` class analogous to the` NearestCentroid` class with `sklearn`**

In [ ]:
class MinimumDistanceClassifier():
    #wyznacza elementy reprezentatywne
    def fit(self, train, train_labels):
        pass
    #wyznacza klasę dla każdej danej testowej
    def predict(self, test):
        pass
    #wyznacza dokładność klasyfikatora
    def score(self, test, test_labels):
        pass
    ### BEGIN SOLUTION
    def fit(self, train, train_labels):
        self.class_means = np.vstack([ np.mean( train[np.ravel(train_labels==i)], axis=0) \
                                                              for i in np.unique(train_labels) ])
    
    def predict(self, test):
        dists = np.stack([ np.sqrt(np.sum( (test - self.class_means[i])**2, axis=1)) \
                                                                  for i in range(len(self.class_means)) ]).T
        return np.argmin(dists, axis=1)
    
    def score(self, test, test_labels):
        return np.sum(self.predict(test)==test_labels)/len(test_labels)
    
    ###END SOLUTION
        

In [ ]:
clf = MinimumDistanceClassifier()
clf.fit(train, train_labels)
clf.score(test, test_labels)
np.testing.assert_almost_equal(score, clf.score(test, test_labels))

**9\. Porównanie z klasyfikatorem `NearestCentroid` z `sklearn`**

**9 \. Comparison with the `NearestCentroid` classifier with` sklearn`**

In [ ]:
from sklearn.neighbors import NearestCentroid 

ncent = NearestCentroid()
ncent.fit(train, train_labels)
np.testing.assert_array_almost_equal(ncent.predict(test),  clf.predict(test))
assert ncent.score(test, test_labels) == clf.score(test, test_labels)


In [ ]:
h = .02
x_min, x_max = train[:, 0].min() - 1, train[:, 0].max() + 1
y_min, y_max = train[:, 1].min() - 1, train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
Z = Z.reshape(xx.shape)

plt.pcolormesh(xx, yy, Z, alpha=0.1)
plt.scatter(train[:,0], train[:,1],c= np.ravel(train_labels))
plt.plot(class_means[0,0], class_means[0,1], 'ro',markersize=10)
plt.plot(class_means[1,0], class_means[1,1], 'ro',markersize=10)
plt.plot(class_means[2,0], class_means[2,1], 'ro',markersize=10)